In [171]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
#get the current dataset

data = pd.read_csv("../opensources_fakenewscorpus_balancedtest/opensources_fakenewscorpus_balancedtest.csv")
data.head(10)

,Label,Sentence
0,1,The phoney war was over today with Prime Minis...
1,1,The possibility that we earthlings are not tru...
2,1,With only a few days left for the official clo...
3,1,"‘Rules Are Rules,’ Say Those With Deeply Ingra..."
4,1,MONTREAL — Canadian indie-rock icons Arcade Fi...
5,1,There's still life in #Hollywood after the Osc...
6,1,You too could live in a city where the roads a...
7,1,"Recently, #Niantic has been introducing change..."
8,1,Back to article: Donald Trump melts down on Tw...
9,1,"Your problems solved, with Holly Harper\n\nDea..."


In [172]:
liar_data = pd.read_csv("train.tsv", sep="\t")

first_val = liar_data.columns
liar_data.loc[-1] = first_val
liar_data.index = liar_data.index + 1  
liar_data = liar_data.sort_index()

liar_data.drop(columns=liar_data.columns[3:], inplace=True)
liar_data.columns = ["source", "label", "sentence"]
liar_data.drop(columns=["source"], inplace=True)

# propaganda = liar_data.loc[liar_data["label"] == "half-true"]
# propaganda2 = liar_data.loc[liar_data["label"] == "mostly-false"] #no mostly false
# hoax = liar_data.loc[liar_data["label"] == "false"]

a = liar_data.groupby("label").count()
a

,sentence
label,
barely-true,1654
false,1995
half-true,2114
mostly-true,1962
pants-fire,839
true,1676


Half true/Mostly false -> propaganda\
false/pants fire -> hoax\
Mostly false doesnt exist, maybe was referring to barely true

In [173]:
combined_propaganda = pd.DataFrame(columns=["Label", "Sentence"])
combined_hoax = pd.DataFrame(columns=["Label", "Sentence"])

for file in ["news-a.tsv", "tweets.tsv", "test.tsv", "train.tsv", "valid.tsv"]:
    new_data = pd.read_csv(file, sep="\t")

    first=new_data.columns[-1]
    new_data.drop(columns=new_data.columns[3:], inplace=True)
    new_data.columns = ["source", "Label", "Sentence"]
    new_data.loc[-1] = ['News', 0, first]  
    new_data.index = new_data.index + 1  
    new_data = new_data.sort_index()  

    propaganda_filter1 = (new_data['Label'] == 1)
    propaganda_filter2 = (new_data['Label'] == "half-true")
    propaganda_filter3 = (new_data['Label'] == "barely-true")
    hoax_filter1 = (new_data['Label'] == "false")
    hoax_filter2 = (new_data['Label'] == "pants-fire")
    propaganda = new_data.loc[propaganda_filter1 | propaganda_filter2 | propaganda_filter3]
    hoax = new_data.loc[hoax_filter1 | hoax_filter2]
    del propaganda["source"]
    del hoax["source"]

    combined_propaganda = pd.concat([combined_propaganda, propaganda], ignore_index=True)
    combined_hoax = pd.concat([combined_hoax, hoax], ignore_index=True)


print(combined_hoax.shape) #Not enough present in liar
print(combined_propaganda.shape) #Enough to completely replace the opensources dataset

(3553, 2)
(17591, 2)


In [174]:
data = pd.read_csv("../opensources_fakenewscorpus_balancedtest/opensources_fakenewscorpus_balancedtest.csv")

#replace the values in the dataset
propaganda = data.loc[data['Label'] == 3] #propaganda
hoax = data.loc[data['Label'] == 2] #hoax

#get the old hoax and remove enough samples to add new hoax
# hoax_sample = hoax.sample(15000-combined_hoax.shape[0])
# hoax_new = pd.concat([hoax_sample, combined_hoax], ignore_index=True)
combined_propaganda["Label"] = 3
combined_hoax['Label'] = 2

#remove the old hoax and propaganda from the dataframe
data.drop(data[(data["Label"] == 3) | (data["Label"] == 2)].index, inplace=True)

data = pd.concat([data, combined_propaganda, combined_hoax], ignore_index=True)
data = data.sort_values(by=["Label"])
data.index = range(data.shape[0])


In [175]:
data.groupby("Label").count()

,Sentence
Label,
1,15000
2,3553
3,17591
4,15000


In [176]:
x = pd.DataFrame(data["Sentence"])
y = pd.DataFrame(data["Label"])

sampler = RandomUnderSampler(sampling_strategy={1: 3500, 2: 3500, 3: 3500, 4: 3500}, random_state=10)
undersampled_x, undersampled_y  = sampler.fit_resample(x, y)

out = pd.concat([undersampled_y,undersampled_x], axis = 1)
out.index = range(out.shape[0])
out.columns = ["Label", "Sentence"]
out

,Label,Sentence
0,1,Overpaid council boss refreshingly open about ...
1,1,"""Hazing is an important part of joining any ex..."
2,1,A former #doctor with the #Bronx-Lebanon #Hosp...
3,1,"Pop music sensation, #Justin Bieber, 23, has h..."
4,1,DISAPPOINTED Daniel Bryan fans have been asked...
...,...,...
13995,4,"• 3 min read \nThursday, the Golden State Warr..."
13996,4,"Articulating the rule he hopes to stick by, Mr..."
13997,4,"The Mets need a new catcher, a starting left f..."
13998,4,Photo\n\nIt hasn’t quite been two decades sinc...


In [177]:
out.to_csv('opensources_fakenewscorpus_modified_undersampled.csv', index=False, header=False)